## Advanced Problems – Python Properties

This notebook contains a set of advanced exercises focused on Python's `@property` mechanism.

- First, try to solve the problems in the **Problems** section.
- Then, compare your work with the provided code in the **Solutions** section.

All exercises are designed to encourage good practices:
- Use private backing attributes (with a leading underscore).
- Avoid code duplication by reusing validation helpers.
- Raise appropriate exceptions (`TypeError`, `ValueError`, `AttributeError`).
- Keep properties side-effect free (no printing or I/O inside getters/setters).


In [1]:
import math
import hashlib


# Problems


### Problem 1 – Bidirectional temperature properties

Create a `Temperature` class that stores the internal temperature in degrees Celsius, but exposes *two* properties:

- `celsius` (read/write)
- `fahrenheit` (read/write)

Requirements:

1. The internal storage must always be in Celsius (for example, in a private attribute `_celsius`).
2. Setting `celsius` must update the internal Celsius value.
3. Setting `fahrenheit` must convert that value to Celsius and store the converted Celsius value internally.
4. Reading either property should always return a float.
5. Temperatures lower than absolute zero (−273.15°C) must be rejected with a `ValueError`.
6. Non-numeric values must be rejected with a `TypeError`.

Example usage:

```python
t = Temperature(0)        # 0 °C
t.fahrenheit              # 32.0
t.fahrenheit = 212        # 100 °C
t.celsius                 # 100.0
```


In [2]:
# TODO: Implement the Temperature class according to the specification above.

class Temperature:
    def __init__(self, celsius):
        pass

    # define celsius and fahrenheit properties here


### Problem 2 – Rectangle with dependent properties

Create a `Rectangle` class with:

- `width` (read/write)
- `height` (read/write)
- `area` (read-only)
- `perimeter` (read-only)
- `is_square` (read/write)

Requirements:

1. Store the dimensions in private attributes (for example, `_width` and `_height`).
2. `width` and `height` must be positive numbers (not zero or negative). Invalid values should raise:
   - `TypeError` for non-numeric values.
   - `ValueError` for non-positive values.
3. `area` and `perimeter` are computed from `width` and `height` and are read-only.
4. `is_square` returns `True` if the rectangle is a square, `False` otherwise.
5. Setting `is_square` to `True` should make the rectangle a square by adjusting the `height` to match the current `width`.
6. Setting `is_square` to `False` should **not** change the dimensions; just ensure the value is a boolean (and raise `TypeError` otherwise).

Example usage:

```python
r = Rectangle(2, 4)
r.area          # 8.0
r.perimeter     # 12.0
r.is_square     # False
r.is_square = True
r.width, r.height, r.is_square   # (2.0, 2.0, True)
```


In [3]:
# TODO: Implement the Rectangle class according to the specification above.

class Rectangle:
    def __init__(self, width, height):
        pass

    # define width, height, area, perimeter, is_square properties here


### Problem 3 – Fixing recursive properties and adding validation

Consider the following buggy implementation of a `Person` class:

```python
class Person:
    def __init__(self, name):
        self.name = name

    @property
    def name(self):
        return self.name

    @name.setter
    def name(self, value):
        self.name = value
```

This implementation will cause infinite recursion when accessing or setting `name`.

Tasks:

1. Rewrite the `Person` class so that it uses a private backing attribute (for example, `_name`).
2. Add validation in the setter so that:
   - `name` must be a string, otherwise raise `TypeError`.
   - `name` cannot be an empty or whitespace-only string, otherwise raise `ValueError`.
3. The getter simply returns the stored name.

Do not change the public interface: code should still be able to use `person.name` and `person.name = ...`.


In [4]:
# TODO: Rewrite the Person class to avoid recursion and add validation.

class Person:
    def __init__(self, name):
        pass

    @property
    def name(self):
        pass

    @name.setter
    def name(self, value):
        pass


### Problem 4 – Lazy (on-demand) computation with caching

Sometimes computing a value is expensive, and we do not want to do the work until the value is actually needed. We also want to cache the result so that subsequent access is cheap.

Create a class `ExpensiveData` with:

- A constructor that accepts a `source` string (for example, a file path or URL). Store it in a private attribute `_source`.
- A read/write `source` property.
- A read-only `data` property that loads and caches data on first access.

Requirements:

1. The internal cache should be stored in a private attribute (for example, `_data`) and initialized to `None`.
2. The `data` property should:
   - If `_data` is `None`, call a private method (for example, `_load_data`) to compute the data and store it in `_data`.
   - Return the cached `_data` value.
3. Changing `source` via the `source` property should automatically invalidate the cache by setting `_data` back to `None`.
4. For this exercise, you can simulate `_load_data` rather than actually reading a file. For example, `_load_data` can return a list of uppercased characters of the `source` string.

Example usage:

```python
e = ExpensiveData("abc")
first = e.data   # triggers a load
second = e.data  # uses the cached value
e.source = "xyz" # invalidates the cache
third = e.data   # triggers a new load
```


In [5]:
# TODO: Implement ExpensiveData with lazy, cached data loading.

class ExpensiveData:
    def __init__(self, source):
        pass

    # define source and data properties, and a private _load_data helper here


### Problem 5 – Write-only password property

Create a `User` class that handles passwords correctly using properties.

Requirements:

1. The constructor accepts a `username` string and a `password` string.
2. The raw password must **never** be stored directly. Instead, store only a password hash in a private attribute (for example, `_password_hash`).
3. Implement a write-only `password` property:
   - Setting `user.password = "something"` should compute and store a hash in `_password_hash`.
   - Reading `user.password` should raise `AttributeError` with a clear message that the password is write-only.
4. Use a private method (for example, `_hash_password`) that takes a string and returns a hashed string. For this exercise, you can use `hashlib.sha256` on the UTF-8 encoded password.
5. Implement a `check_password(candidate: str) -> bool` method that hashes the candidate and compares it to the stored hash.
6. Add basic type checking in the setter and hashing method so that non-string passwords raise `TypeError`.

Example usage:

```python
u = User("alice", "secret")
u.check_password("secret")      # True
u.check_password("wrong")       # False

u.password = "new-secret"
u.check_password("new-secret")  # True
```


In [6]:
# TODO: Implement the User class with a write-only password property.

class User:
    def __init__(self, username, password):
        pass

    # define password property and check_password method here


# Solutions

Below are reference implementations for the problems above. Try to solve the problems yourself before looking at the solutions.


## Solution 1 – Temperature


In [7]:
class Temperature:
    """Represents a temperature stored internally in degrees Celsius.

    Provides bidirectional celsius and fahrenheit properties with validation.
    """

    ABSOLUTE_ZERO_C = -273.15

    def __init__(self, celsius):
        # Initialize backing attribute and delegate to the property for validation.
        self._celsius = 0.0
        self.celsius = celsius

    @staticmethod
    def _validate_celsius(value):
        if not isinstance(value, (int, float)):
            raise TypeError("Temperature must be a number.")
        value = float(value)
        if value < Temperature.ABSOLUTE_ZERO_C:
            raise ValueError("Temperature cannot be below absolute zero.")
        return value

    @property
    def celsius(self):
        """Temperature in degrees Celsius."""
        return self._celsius

    @celsius.setter
    def celsius(self, value):
        self._celsius = self._validate_celsius(value)

    @property
    def fahrenheit(self):
        """Temperature in degrees Fahrenheit."""
        return self._celsius * 9.0 / 5.0 + 32.0

    @fahrenheit.setter
    def fahrenheit(self, value):
        if not isinstance(value, (int, float)):
            raise TypeError("Temperature must be a number.")
        value = float(value)
        celsius = (value - 32.0) * 5.0 / 9.0
        self._celsius = self._validate_celsius(celsius)


## Solution 2 – Rectangle


In [8]:
class Rectangle:
    """Axis-aligned rectangle with width and height.

    Provides computed area, perimeter, and an is_square property.
    """

    def __init__(self, width, height):
        # Initialize backing attributes and use properties for validation.
        self._width = 0.0
        self._height = 0.0
        self.width = width
        self.height = height

    @staticmethod
    def _validate_dimension(value, name):
        if not isinstance(value, (int, float)):
            raise TypeError(f"{name} must be a number.")
        value = float(value)
        if value <= 0:
            raise ValueError(f"{name} must be strictly positive.")
        return value

    @property
    def width(self):
        return self._width

    @width.setter
    def width(self, value):
        self._width = self._validate_dimension(value, "width")

    @property
    def height(self):
        return self._height

    @height.setter
    def height(self, value):
        self._height = self._validate_dimension(value, "height")

    @property
    def area(self):
        return self.width * self.height

    @property
    def perimeter(self):
        return 2.0 * (self.width + self.height)

    @property
    def is_square(self):
        return math.isclose(self.width, self.height)

    @is_square.setter
    def is_square(self, value):
        if not isinstance(value, bool):
            raise TypeError("is_square must be set to a boolean.")
        if value:
            # Adjust height so that the rectangle becomes a square.
            self._height = self._width
        # If value is False, we do not change dimensions; caller can adjust
        # width and height directly using their respective properties.


## Solution 3 – Person (fixing recursion and adding validation)


In [9]:
class Person:
    """Person with a validated name property.

    Avoids recursive property access by using a private backing attribute.
    """

    def __init__(self, name):
        # Initialize backing attribute and delegate to the property for validation.
        self._name = ""
        self.name = name

    @property
    def name(self):
        return self._name

    @name.setter
    def name(self, value):
        if not isinstance(value, str):
            raise TypeError("name must be a string.")
        value = value.strip()
        if not value:
            raise ValueError("name cannot be empty or whitespace only.")
        self._name = value


## Solution 4 – ExpensiveData with lazy caching


In [10]:
class ExpensiveData:
    """Simulates an expensive data source with lazy loading and caching.

    Changing the source invalidates the cached data.
    """

    def __init__(self, source):
        self._source = None
        self._data = None
        self.source = source

    @property
    def source(self):
        return self._source

    @source.setter
    def source(self, value):
        if not isinstance(value, str):
            raise TypeError("source must be a string.")
        self._source = value
        # Invalidate cache whenever source changes.
        self._data = None

    def _load_data(self):
        """Simulate an expensive computation.

        For this exercise, we simply return a list of uppercased
        characters from the source string.
        """
        return [ch.upper() for ch in self._source]

    @property
    def data(self):
        # Lazy load: compute only on first access or after cache invalidation.
        if self._data is None:
            self._data = self._load_data()
        return self._data


## Solution 5 – User with write-only password property


In [11]:
class User:
    """User with a hashed, write-only password property."""

    def __init__(self, username, password):
        if not isinstance(username, str):
            raise TypeError("username must be a string.")
        self.username = username
        self._password_hash = ""
        # Delegate to the property so we reuse validation and hashing logic.
        self.password = password

    @staticmethod
    def _hash_password(raw_password):
        if not isinstance(raw_password, str):
            raise TypeError("password must be a string.")
        # NOTE: This is for demonstration only. Production code would need salt,
        # key stretching, and so on.
        return hashlib.sha256(raw_password.encode("utf-8")).hexdigest()

    @property
    def password(self):
        # Do not allow reading the raw password.
        raise AttributeError("Password is write-only and cannot be read.")

    @password.setter
    def password(self, raw_password):
        self._password_hash = self._hash_password(raw_password)

    def check_password(self, candidate):
        """Return True if candidate matches the stored password hash."""
        return self._hash_password(candidate) == self._password_hash


# Tests

The following cells contain basic tests for each of the solutions above. Run them to verify that the implementations behave as expected.


## Tests for Temperature


In [12]:
# Basic tests for Temperature

t = Temperature(0)
assert isinstance(t.celsius, float)
assert isinstance(t.fahrenheit, float)
assert math.isclose(t.celsius, 0.0)
assert math.isclose(t.fahrenheit, 32.0)

t.fahrenheit = 212
assert math.isclose(t.celsius, 100.0)
assert math.isclose(t.fahrenheit, 212.0)

# Absolute zero and type validation
try:
    Temperature(-300)
except ValueError:
    pass
else:
    raise AssertionError("Expected ValueError for below absolute zero")

for bad in ("cold", None, object()):
    try:
        Temperature(bad)
    except TypeError:
        pass
    else:
        raise AssertionError("Expected TypeError for non-numeric temperature")

print("Temperature tests passed.")


Temperature tests passed.


## Tests for Rectangle


In [13]:
# Basic tests for Rectangle

r = Rectangle(2, 4)
assert math.isclose(r.width, 2.0)
assert math.isclose(r.height, 4.0)
assert math.isclose(r.area, 8.0)
assert math.isclose(r.perimeter, 12.0)
assert r.is_square is False

r.is_square = True
assert math.isclose(r.width, 2.0)
assert math.isclose(r.height, 2.0)
assert r.is_square is True

# Dimension validation
for bad in (0, -1):
    try:
        Rectangle(bad, 1)
    except ValueError:
        pass
    else:
        raise AssertionError("Expected ValueError for non-positive width")

for bad in ("wide", None):
    try:
        Rectangle(bad, 1)
    except TypeError:
        pass
    else:
        raise AssertionError("Expected TypeError for non-numeric width")

try:
    r.is_square = 1
except TypeError:
    pass
else:
    raise AssertionError("Expected TypeError when setting is_square to non-bool")

print("Rectangle tests passed.")


Rectangle tests passed.


## Tests for Person


In [14]:
# Basic tests for Person

p = Person(" Alice ")
assert p.name == "Alice"

p.name = "Bob"
assert p.name == "Bob"

try:
    p.name = "   "
except ValueError:
    pass
else:
    raise AssertionError("Expected ValueError for empty/whitespace name")

for bad in (None, 123, object()):
    try:
        p.name = bad
    except TypeError:
        pass
    else:
        raise AssertionError("Expected TypeError for non-string name")

print("Person tests passed.")


Person tests passed.


## Tests for ExpensiveData


In [15]:
# Basic tests for ExpensiveData

e = ExpensiveData("abc")
first = e.data
second = e.data

# Same object (cached) and expected transformation
assert first is second
assert first == ["A", "B", "C"]

# Changing source invalidates cache
e.source = "xyz"
third = e.data
assert third == ["X", "Y", "Z"]
assert third is not first

try:
    e.source = 123
except TypeError:
    pass
else:
    raise AssertionError("Expected TypeError for non-string source")

print("ExpensiveData tests passed.")


ExpensiveData tests passed.


## Tests for User


In [16]:
# Basic tests for User

u = User("alice", "secret")
assert u.check_password("secret") is True
assert u.check_password("wrong") is False

# Password should be write-only
try:
    _ = u.password
except AttributeError:
    pass
else:
    raise AssertionError("Expected AttributeError when reading password")

# Changing password should change the hash and update check_password
old_hash = u._password_hash
u.password = "new-secret"
assert u._password_hash != old_hash
assert u.check_password("new-secret") is True
assert u.check_password("secret") is False

try:
    u.password = 123
except TypeError:
    pass
else:
    raise AssertionError("Expected TypeError for non-string password")

print("User tests passed.")


User tests passed.
